In [49]:
!node --version
!pip3 install kaggle-environments -U
from kaggle_environments import make
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys
import numpy as np
from numpy import unravel_index

v12.13.1


In [50]:
# CONSTANTS
CITY_TO_RESOURCE_MULTIPLIER = 10
CITY_TO_RESOURCE_MAX_RANGE = 15
CITY_TO_CITY_MULTIPLIER = 100

In [51]:
def create_city_heatmap(game_state, player):
    width, height = game_state.map_width, game_state.map_height
    heatmap = [[0 for y in range(width)] for x in range(width)]
    for y in range(height):
        for x in range(width):
            curPos = Position(x,y)
            city_positions = get_positions_within_distance(curPos, game_state)
            res_positions = get_positions_within_distance(curPos, game_state, CITY_TO_RESOURCE_MAX_RANGE)
            for i in res_positions:
                cell = game_state.map.get_cell_by_pos(i)
                if cell.has_resource():
                    heatmap[x][y] +=  CITY_TO_RESOURCE_MULTIPLIER / (curPos.distance_to(i) + 1)
            for i in city_positions:
                cell = game_state.map.get_cell_by_pos(i)
                if cell.citytile is not None and cell.citytile.team == player.team:
                    heatmap[x][y] += CITY_TO_CITY_MULTIPLIER
    return heatmap
# put a score to every tile based off
# of distance from resources, unit, and adjacency to cities
def find_best_city_to_build(pos, game_state, player):
    heatmap = create_city_heatmap(game_state, player)
    max_val = -1
    max_ind = [0,0]
    for x in range(len(heatmap)):
        for y in range(len(heatmap[x])):
            if heatmap[x][y] > max_val and is_legal_to_build(Position(x,y),game_state):
                max_ind = [x,y]
                max_val = heatmap[x][y]
    return Position(max_ind[0],max_ind[1])
#get max of create_city_heatmap
def is_legal_to_build(pos, game_state):
    cell = game_state.map.get_cell_by_pos(pos)
    if not cell.has_resource() and cell.citytile is None:
        return True
    return False
def move_unit(pos, dest):
    pass
#collision safe move


In [52]:
def get_positions_within_distance(pos, game_state,dist = 1):
    max_width, max_height = game_state.map_width, game_state.map_height
    retVal = []
    for i in range(-dist, dist+1):
        temp = dist - abs(i)
        for j in range(-temp,temp+1):
            x = pos.x + i
            y = pos.y + j
            if 0 <= x < max_width and  0 <= y < max_height:
                retVal.append(Position(x,y))
    del retVal[0] # don't return the start tile
    return retVal



In [53]:
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles
def find_empty_tiles(game_state):
    empty_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if not cell.has_resource() and cell.citytile == None:
                empty_tiles.append(cell)
    return empty_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile
def find_closest_empty_tile(pos, empty_tiles):
    closest_empty_tile = None
    closest_dist = math.inf
    for empty_tile in empty_tiles:
        dist = empty_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_empty_tile = empty_tile
    return closest_empty_tile

def count_total_city_tiles(player):
    count = 0
    for city_id, city in player.cities.items():
        count += len(city.citytiles)
    return count
        

game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)
    empty_tiles = find_empty_tiles(game_state)
    
    for city_id, city in player.cities.items():
        for city_tile in city.citytiles:
            if city_tile.can_act():
                action = city_tile.build_worker()
                actions.append(action)
    
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():   
            # we want to mine only if there is space left in the worker's cargo
            if unit.get_cargo_space_left() > 0:
                # find the closest resource if it exists to this unit
                closest_resource_tile = find_closest_resources(unit.pos, player, resource_tiles)
                if closest_resource_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_resource_tile.pos))
                    actions.append(action)
            else:
                if(len(player.units) >= count_total_city_tiles(player)):
                    closest_empty_tile = find_closest_empty_tile(unit.pos, empty_tiles)
                    if closest_empty_tile is not None:
                        if unit.pos == closest_empty_tile.pos:
                            action = unit.build_city()
                        else:
                            action = unit.move(unit.pos.direction_to(closest_empty_tile.pos))
                        actions.append(action)
                else:
                # find the closest citytile and move the unit towards it to drop resources to a citytile to fuel the city
                    closest_city_tile = find_closest_city_tile(unit.pos, player)
                    if closest_city_tile is not None:
                        # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                        action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                        actions.append(action)
    pos = find_best_city_to_build("blank", game_state, player)
    actions.append(annotate.circle(pos.x,pos.y))
    return actions

In [54]:
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, "simple_agent"])
env.render(mode="ipython", width=1200, height=800)

[WARN] (match_DzX02n317z4b) - Agent 0 tried to build unit on tile (3, 27) but unit cap reached. Build more CityTiles!; turn 0; cmd: bw 3 27
[WARN] (match_DzX02n317z4b) - Agent 0 tried to build unit on tile (3, 27) but unit cap reached. Build more CityTiles!; turn 1; cmd: bw 3 27
[WARN] (match_DzX02n317z4b) - Agent 0 tried to build unit on tile (3, 27) but unit cap reached. Build more CityTiles!; turn 2; cmd: bw 3 27
[WARN] (match_DzX02n317z4b) - Agent 0 tried to build unit on tile (3, 27) but unit cap reached. Build more CityTiles!; turn 3; cmd: bw 3 27
[WARN] (match_DzX02n317z4b) - Agent 0 tried to build unit on tile (3, 27) but unit cap reached. Build more CityTiles!; turn 4; cmd: bw 3 27
[WARN] (match_DzX02n317z4b) - Agent 0 tried to build unit on tile (3, 27) but unit cap reached. Build more CityTiles!; turn 5; cmd: bw 3 27
[WARN] (match_DzX02n317z4b) - Agent 0 tried to build unit on tile (4, 26) but unit cap reached. Build more CityTiles!; turn 6; cmd: bw 4 26
[WARN] (match_DzX02n

In [55]:
%%writefile agent.py
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math

# All code goes here

Overwriting agent.py


In [56]:
!tar -czf submission.tar.gz *

tar: submission.tar.gz: Can't add archive to itself
